This notebook preprocesses data and saves the preprocessed data to csv files

In [ ]:
from google.colab import drive
import os 
drive.mount('/gdrive/')

Mounted at /gdrive/


In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import datetime
import holidays

In [ ]:
def get_holiday(year):
  # have a list of holidays
  holiday_days = set()

  # Print all the holidays in US in year 2016
  for ptr in holidays.US(years = year).items():
      #print(ptr[0], ptr[1])
      holiday_days.add(ptr[0])

  # further processing to include several days before and after

  # christmas - new year 
  start_date = date(year, 12, 24) 
  end_date = date(year, 12, 31)  

  delta = end_date - start_date   # returns timedelta

  for i in range(delta.days + 1):
      day = start_date + timedelta(days=i)
      holiday_days.add(day)

  # After new year -> until the first monday
  # specify year and month
  yearMonth = str(year) + '-01'
  firstDay = str(year) + '-01-01'
  firstDate = date(year,1,1)
  # getting date of first monday
  firstMonday = np.busday_offset(yearMonth, 0, 
                            roll='forward', 
                            weekmask='Mon')
  # if firstDay is not Monday
  if str(firstMonday) != firstDay:
    start_date = date(year, 1, 1) 
    end_date = datetime.datetime.strptime(str(firstMonday),'%Y-%m-%d').date()-timedelta(days=1) 

    delta = end_date - start_date   # returns timedelta

    for i in range(delta.days + 1):
        day = start_date + timedelta(days=i)
        holiday_days.add(day)

  # if firstDay is Monday, only include that day-> already in the auto-generated holiday list

  # july 4th: one week 
  indepDay = date(year,7,4)
  prev_monday = indepDay + timedelta(days=-indepDay.weekday())
  next_sunday = indepDay + timedelta(days=-(indepDay.weekday()+1), weeks=1)
  delta = next_sunday - prev_monday   # returns timedelta
  for i in range(delta.days + 1):
    day = prev_monday + timedelta(days=i)
    holiday_days.add(day)

  return list(holiday_days)

In [ ]:
def is_holiday(year,row):

  holiday_list = get_holiday(year)

  dow = row['date'].weekday() 
  # 5 Sat, 6 Sun
  if dow == 5 or dow == 6: 
    return 1
  # Mon-Thursday
  elif dow < 4:
    if row['date'] not in holiday_list:
      return 0
    else:
      return 1
  # if Friday: after dusk, holiday = 1; before dusk, holiday list
  else:
    if row['light_cat'] == 'dark' or row['date'] in holiday_list:
      return 1
    else:
      return 0

In [ ]:
def remove_duplicates(row):
  if row['speeding_only'] == 'speeding-repeated_entries':
    return row['all_violation'][0].strip()
  else:
    return row['violation']

def exclusive(vio_lst):
  count = 0
  for vio in vio_lst:
    # count number of violations containing 'speed'
    if 'speed' in vio:
      count += 1
  # if only 1 speeding-related violation, return 1
  # if all of the violations (for this record) include speeding and more than 1 violation, return 2
  # if include violations other than speeding, return 3
  if count == 1 and count == len(vio_lst):
    return 'speeding-1'
  elif count == len(vio_lst):
    if len(vio_lst) == len(set(vio_lst)):
      return 'speeding-repeated_entries'
    return 'speeding-multiple'
  elif count < len(vio_lst):
    return 'speeding+others'

In [ ]:
def speeding_filter(df):
  print('Before speed filtering: ', len(df))
  df['violation'] = [s.lower() for s in df['violation']]
  df = df.loc[df['violation'].str.contains('speed', regex = False),:]
  df['violation'] = df['violation'].map(lambda x: x.replace('(#)',''))
  df['violation'] = df['violation'].map(lambda x: x.strip())

  # get a list of violations for each record, and apply self-defined func exclusive
  df['all_violation'] = df['violation'].str.split('|')
  df['speeding_only'] = df['all_violation'].map(lambda x: exclusive(x))

  # filter out rows with violations other than speeding
  df = df.loc[df['speeding_only'].isin(['speeding-repeated_entries','speeding-1'])]
  # print('All single speeding violation records: ', len(df))
  df['violation'] = df.apply(lambda row: remove_duplicates(row), axis = 1)

  df = df.loc[~df['violation'].isin(['speeding-10% or more above posted speed','unsafe speed','fail to control speed','speeding cmv 15 mph or more over limit']),:]
  # df.drop(['all_violation','speeding_only','violation'], axis = 1, inplace = True)
  print('After speed filtering: ', len(df))
  return df

In [ ]:
! pip install astral
! pip install PyAstronomy

     |████████████████████████████████| 711 kB 5.1 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
  Created wheel for PyAstronomy: filename=PyAstronomy-0.17.1-py3-none-any.whl size=504451 sha256=29d8f57517b308c86b483354b9d319d5b1ef3a22bd086561c080f0d2e227a26a
  Stored in directory: /root/.cache/pip/wheels/0d/3b/34/d1ade1c2c660da472dd7e441b19eaf1066cc778f65fdcec5d4
  Created wheel for quantities: filename=quantities-0.13.0-py3-none-any.whl size=77860 sha256=cf238d39e75ce1eb9baeb7b4007a5616442c66224bcd5d6d9e2d5f304e5968f5
  Stored in directory: /root/.cache/pip/wheels/c1/9d/0a/cbfeaa29541b7fc8752d35f368e63578a9f64229878841412a
Successfully built PyAstronomy quantities


In [ ]:
from PyAstronomy import pyasl
from astral import LocationInfo
from astral.sun import sun
import pytz
import datetime
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def sunrise_sunset(row,info):
  """
  lat and lnt: latitude and longitude, float
  dat: datetime.date object
  info: 'sunrise';'sunset';'dawn';'dusk'
  return: return the datetime.datetime object for specified info type
  """
  #sunrise_sunset(lat,lng,dat,rise_set)
  lat = row['lat']
  lng = row['lng']
  dat = row['date']

  # use central timezone, need to change for two counties!!!!
  city = LocationInfo(timezone = 'US/Central', latitude = lat, longitude = lng)
  s = sun(city.observer, date = dat, tzinfo = city.timezone)

  # Get relevant info
  if info == 'sunset':
    # central time zone
    sunset = s["sunset"]
    sunset = sunset.strftime('%Y:%m:%d:%H:%M:%S')
    sunset = datetime.datetime.strptime(sunset, '%Y:%m:%d:%H:%M:%S')
    return sunset
   
  if info == 'sunrise':
    # central time zone
    sunrise = s["sunrise"]
    sunrise = sunrise.strftime('%Y:%m:%d:%H:%M:%S')
    sunrise = datetime.datetime.strptime(sunrise, '%Y:%m:%d:%H:%M:%S')
    return sunrise
    
  if info == 'dawn':
    # central time zone
    dawn = s["dawn"]
    dawn = dawn.strftime('%Y:%m:%d:%H:%M:%S')
    dawn = datetime.datetime.strptime(dawn, '%Y:%m:%d:%H:%M:%S')
    return dawn

  if info == 'dusk':
    # central time zone
    dusk = s["dusk"]
    dusk = dusk.strftime('%Y:%m:%d:%H:%M:%S')
    dusk = datetime.datetime.strptime(dusk, '%Y:%m:%d:%H:%M:%S')
    return dusk

Split time of day into three categories:
- Midnight to Before dawn = dark
- dawn to 30 min after sunrise = half-light
- 30 min after sunrise to 30 min before sunset = light
- 30 min before sunset to dusk = half-light
- Dusk to midnight = dark

In [ ]:
def get_light_cat(row):
  delta = datetime.timedelta(minutes = 30)
  light_lower = row['sunrise'] + delta
  light_higher = row['sunset'] - delta

  if row['stop_time'] < row['dawn'] or row['stop_time'] > row['dusk']:
    return 'dark'
  if light_lower <= row['stop_time'] <= light_higher:
    return 'light'
  else:
    return 'half_light'

In [ ]:
def add_daylight(df):
  """
  INPUT:
  df: traffic stop dataframe
  county_info: county->lat/lng dataframe
  OUTPUT: 
  dataframe with a new column of daytime: 1 if daytime 0 if 
  NOTE:
  if a row(records) does not have either county info or latitude/longitude info, it will be deleted
  """
  # read in county_info
  county_info = pd.read_csv('/gdrive/MyDrive/traffic_stop/table_county.csv')
  
  # preprocessing county_info dataframe
  # minus sign, delete celsius sign
  county_info['Longitude'] = county_info['Longitude'].map(lambda x: '-' + x[1:])
  for col in ['Latitude','Longitude']:
    county_info[col] = county_info[col].map(lambda x: x[:-1])
    county_info[col] = county_info[col].astype('float')

  # we are focusing on Texas
  county_info = county_info.loc[county_info['State'] == 'TX',:]

  # 1) select rows with missing values in latitude/longitude, but having county info -> needs processing
  cols = ['lat','lng']
  mask = df[cols].isna().any(axis=1)
  df_c = df[mask]
  df_c = df_c.loc[df_c['county_name'].notna(),:]

  # 2) select rows with latitude & longitude values
  df_complete = df.loc[(df['lat'].notna())&(df['lng'].notna())]

  # make a 'new_county' column to correspond to the identifiers in the county_info csv
  df_c['new_county'] = df_c['county_name'].map(lambda x: x.replace(' County', ''))
  df_c['new_county'] = df_c['new_county'].replace('Dewitt','DeWitt')

  # rename county info's column
  county_info = county_info[['State','County [2]','Latitude','Longitude']]
  county_info.rename(columns={"County [2]": "county"}, inplace = True)

  # fill in missing latitude and longitude with county info
  df_merged = df_c.merge(county_info, how = 'inner', left_on = 'new_county', right_on = 'county')
  df_merged['lat'].fillna(df_merged['Latitude'], inplace = True)
  df_merged['lng'].fillna(df_merged['Longitude'], inplace = True)

  # merged the two dfs: 2) originally with lat/lng and 1) without but filled based on county info
  df_merged.drop(['new_county','State','county','Latitude','Longitude'], axis = 1, inplace = True)
  df = pd.concat([df_complete,df_merged], ignore_index=True)
  
  #for col in ['lat','lng']:
    #df[col].astype('float')

  # add 'sunrise', 'sunset', 'dawn', 'dusk' columns, data types are all datetime.datetime object
  df['sunset'] = df.apply(lambda row: sunrise_sunset(row,'sunset'),axis=1)
  df['sunrise'] = df.apply(lambda row: sunrise_sunset(row,'sunrise'),axis = 1)
  df['dawn'] = df.apply(lambda row: sunrise_sunset(row,'dawn'),axis = 1)
  df['dusk'] = df.apply(lambda row: sunrise_sunset(row,'dusk'),axis = 1)

  # add 'stop_time' column 
  df['stop_time'] = df.apply(lambda row: datetime.datetime.combine(row['date'], row['time']), axis = 1)

  # get stop time category
  df['light_cat'] = df.apply(lambda row: get_light_cat(row), axis = 1)
 
  return df

In [ ]:
keep_col = ['date', 
                'time', 
                'county_name', 
                'subject_race',
                'subject_sex',
                'violation', 
                'citation_issued',
                'contraband_found',
                'contraband_drugs',
                'contraband_weapons',
                'search_conducted',
                'search_vehicle',
                'lat',
                'lng'
                ]
rm_na_col = ['county_name','violation','subject_race','subject_sex']

def remove_empty_rows(df, colName):
	df = df[df[colName] != 'unknown']
	df = df[df[colName].notna()]
	return(df)

def preprocess(year, keep_col, rm_na_col, violation_type = None):
  # read dataframe
  filename = '/gdrive/MyDrive/traffic_stop/year_data/traffic_' + str(year) + '.parquet'
  df = pd.read_parquet(filename, engine = 'pyarrow')

  # invalid value
  if year == 2013:
    df.drop(df.index[df['lat'] == 74.052879], inplace=True)

  ## drop unrelated columns
  col_drop = [col for col in df.columns if col not in keep_col]
  df.drop(col_drop, axis = 1, inplace = True)

  # remove rows with missing values in rm_na_col
  for col in rm_na_col:
    df = remove_empty_rows(df, col)
  
  # filter speed only include single speed violation with low citation rate
  df = speeding_filter(df)

  ## County names are converted to county type - metropolitan, micropolitan or non-core
  # For definitions, see US OMB website

  # read in county info csv
  county_df = pd.read_csv('/gdrive/MyDrive/traffic_stop/2014-2018.csv')
  county_df = county_df[county_df['State']=='Texas']
  county_df = county_df.filter(items=['Metropolitan Status', 'County Name'])

  # transform column
  df['county'] = [name[:-7] for name in df['county_name']]
  df = df.join(county_df.set_index('County Name'), on='county')
  df.drop('county', 1, inplace=True)
  df.rename(columns={'Metropolitan Status':'county_type'}, inplace=True)

  # !!! need to check where the missing values of county_type come from
  df = df.loc[df['county_type'].notna(),:]

  # Convert 'citation issued' to integer
  df = df.astype({'citation_issued': 'int64'})

  # search and contraband related variables have three levels: None, True, False, if not True, use 0, else 1
  for col in ['contraband_found','contraband_drugs','contraband_weapons','search_conducted','search_vehicle']:
    df[col] = df[col].map({True: 1, False: 0, None: 0})

  # if race is other/unknown, we delete the rows!
  df = df.loc[(df['subject_race'] != 'unknown') & (df['subject_race'] != 'other'),:]

  # add lightning variable
  df = add_daylight(df)
  # add holiday variable
  df['holiday'] = df.apply(lambda row: is_holiday(year, row), axis = 1)

  # df.drop(['sunset','sunrise','dawn','dusk','stop_time','time','lat','lng','date','county_name'], axis = 1, inplace = True)
  
  # get dummies for race and sex
  # df = pd.get_dummies(df)
 
  # Base level: White, Male, Non_core (county type)
  # df.drop(['subject_race_white', 'subject_sex_male', 'county_type_Non core','subject_race_other','subject_race_unknown'], axis = 1, inplace = True)
  print(df.columns)
  return df 

In [ ]:
years = []
for i in range(2006, 2018):
  years.append(i)
years

[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for year in years:
  data_write = preprocess(year=year,keep_col = keep_col, rm_na_col = rm_na_col)
  file_name = 'traffic_' + str(year)
  path = '/gdrive/MyDrive/traffic_stop/year_data_preprocessed/' + file_name + '.parquet'
  data_write.to_parquet(path, engine='pyarrow')

Before speed filtering:  2673511
After speed filtering:  767749
Index(['date', 'time', 'lat', 'lng', 'county_name', 'subject_race',
       'subject_sex', 'violation', 'citation_issued', 'contraband_found',
       'contraband_drugs', 'contraband_weapons', 'search_conducted',
       'search_vehicle', 'all_violation', 'speeding_only', 'county_type',
       'sunset', 'sunrise', 'dawn', 'dusk', 'stop_time', 'light_cat',
       'holiday'],
      dtype='object')
Before speed filtering:  2405744
After speed filtering:  698046
Index(['date', 'time', 'lat', 'lng', 'county_name', 'subject_race',
       'subject_sex', 'violation', 'citation_issued', 'contraband_found',
       'contraband_drugs', 'contraband_weapons', 'search_conducted',
       'search_vehicle', 'all_violation', 'speeding_only', 'county_type',
       'sunset', 'sunrise', 'dawn', 'dusk', 'stop_time', 'light_cat',
       'holiday'],
      dtype='object')
Before speed filtering:  2434970
After speed filtering:  764522
Index(['date', '

ValueError: ignored

In [ ]:
for year in list(range(2016, 2018)):
  data_write = preprocess(year=year,keep_col = keep_col, rm_na_col = rm_na_col)
  file_name = 'traffic_' + str(year)
  path = '/gdrive/MyDrive/traffic_stop/year_data_preprocessed/' + file_name + '.parquet'
  data_write.to_parquet(path, engine='pyarrow')

Before speed filtering:  1832072
After speed filtering:  556284
Index(['date', 'time', 'lat', 'lng', 'county_name', 'subject_race',
       'subject_sex', 'violation', 'citation_issued', 'contraband_found',
       'contraband_drugs', 'contraband_weapons', 'search_conducted',
       'search_vehicle', 'all_violation', 'speeding_only', 'county_type',
       'sunset', 'sunrise', 'dawn', 'dusk', 'stop_time', 'light_cat',
       'holiday'],
      dtype='object')
Before speed filtering:  2197356
After speed filtering:  625507
Index(['date', 'time', 'lat', 'lng', 'county_name', 'subject_race',
       'subject_sex', 'violation', 'citation_issued', 'contraband_found',
       'contraband_drugs', 'contraband_weapons', 'search_conducted',
       'search_vehicle', 'all_violation', 'speeding_only', 'county_type',
       'sunset', 'sunrise', 'dawn', 'dusk', 'stop_time', 'light_cat',
       'holiday'],
      dtype='object')


#### ignore

In [ ]:
#rm_na_col = ['county_name','violation','subject_race','subject_sex']
df =  pd.read_parquet('/gdrive/MyDrive/traffic_stop/year_data/traffic_2016.parquet', engine = 'pyarrow')

In [ ]:
df['subject_sex'].isnull().sum()

113

In [ ]:
sum(df['subject_sex'].value_counts()) + 113

1832207

In [ ]:
len(df)

1832207